In [1]:
#r "nuget: MathNet.Numerics.FSharp, 4.15.0"

open MathNet.Numerics.LinearAlgebra

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package MathNet.Numerics.FSharp version 4.15.0

# Частичная проблема собственных значений
## Варинат 9

In [2]:
let a = matrix [
    [-0.90701; -0.27716; 0.44570]
    [-0.27716; 0.63372; 0.07774]
    [0.44570; 0.07774; -0.95535]
]

**Задание 2.** Найти степенным методом c точностью ε = 0.001 максимальное по модулю собственное число λ матрицы A и соответствующий ему собственный вектор x, так чтобы ||x||_2 = 1

In [3]:
let findEigenIterative accuracy (matrix: Matrix<float>) step = 
    // firstEstimation.[0] не должен быть слишком мал
    let firstEstimation = DenseVector.randomStandard<float> a.RowCount
    firstEstimation.[0] <- 1.
    
    let accurateLambda = matrix.Evd().EigenValues.[a.Evd().EigenValues.AbsoluteMaximumIndex()].Real

    let calculatePosteriorEstimation (eigenValueEstimation: float) (eigenVectorEstimation: Vector<float>) = 
        (matrix * eigenVectorEstimation - eigenValueEstimation * eigenVectorEstimation).L2Norm() /
        eigenVectorEstimation.L2Norm()
    
    let rec loop (previousVector: Vector<float>) iterationTable = 
        // unnormalized
        let y = matrix * previousVector
        let lambda = step y previousVector
        let updatedTable = 
            {|
                K = List.length iterationTable + 1
                Lambda = lambda
                AbsFactError = lambda - accurateLambda |> abs
                AYminusLambdaY = (matrix * y - lambda * y).L2Norm()
                PosteriorEstimation = calculatePosteriorEstimation lambda y
            |}
            |> List.singleton
            |> List.append iterationTable
        
        if calculatePosteriorEstimation lambda y >= accuracy then   
            loop y updatedTable
        else 
            lambda, (y / y.L2Norm()), updatedTable

    loop firstEstimation []

In [4]:
let findEigenPowerMethod accuracy (matrix: Matrix<float>) = 
    findEigenIterative accuracy matrix <| fun (currentVector: Vector<float>) (previousVector: Vector<float>) ->
        // i любое
        (Vector.map2 (/) currentVector previousVector).[0]

In [5]:
let epsilon = 0.001
let (eigenValuePower, eigenVectorPower, iterationTablePower) = findEigenPowerMethod epsilon a
printfn "Наибольшее по модулю собственное число, полученное степенным методом: %f" eigenValuePower
printfn "Соответствующий ему собственный вектор: %O" eigenVectorPower
printfn "Фактическая погрешнсть: %f" <| abs (eigenValuePower - a.Evd().EigenValues.[a.Evd().EigenValues.AbsoluteMaximumIndex()].Real)
iterationTablePower

Наибольшее по модулю собственное число, полученное степенным методом: -1.408069
Соответствующий ему собственный вектор: DenseVector 3-Double
-0,696639
-0,121228
 0,707106

Фактическая погрешнсть: 0.000289


index,AYminusLambdaY,AbsFactError,K,Lambda,PosteriorEstimation
0,2.2939706272973828,0.36709953141458773,1,-1.7748791460884956,0.946508454427856
1,1.2910619630091575,0.2016349183185262,2,-1.2061446963553817,0.3987112177691134
2,0.9671243997890169,0.08876925502755473,3,-1.4965488697014626,0.21494720703580586
3,0.6422691769297507,0.048943511646937,4,-1.3588361030269709,0.10171443157066987
4,0.44056736367309707,0.02142604660734837,5,-1.4292056612812563,0.04959764842954086
5,0.3020823047807478,0.01133042599872347,6,-1.3964491886751844,0.024160951480677515
6,0.2044852606911593,0.005136018561051614,7,-1.4129156332349595,0.01161805978440569
7,0.14054330612146965,0.0026210104773711773,8,-1.4051586041965367,0.0056721956643417505
8,0.09523136085599923,0.0012215231065701992,9,-1.409001137780478,0.0027301551100321303
9,0.06529925535553019,0.0006094081932355166,10,-1.4071702064806724,0.00132978403429513


**Задание 3.** Найти методом скалярных произведений с точностью ε^2 = 0.000001 максимальное по модулю собственное число матрицы λ. Сравнить требуемое количество итераций с п.2.

In [6]:
let findEigenScalarMethod accuracy (matrix: Matrix<float>) = 
    findEigenIterative accuracy matrix <| fun (currentVector: Vector<float>) (previousVector: Vector<float>) ->
        // в предположении, что матрица симметрична
        (currentVector * previousVector) / (previousVector * previousVector)

In [8]:
let epsilon2 = 0.001 ** 2.
let (eigenValueScalar, eigenVectorScalar, iterationTableScalar) = findEigenScalarMethod epsilon2 a
printfn "Наибольшее по модулю собственное число, полученное методом скалярных произведений: %f" eigenValueScalar
printfn "Соответствующий ему собственный вектор: %O" eigenVectorScalar
printfn "Фактическая погрешнсть: %f" <| abs (eigenValueScalar - a.Evd().EigenValues.[a.Evd().EigenValues.AbsoluteMaximumIndex()].Real)
iterationTableScalar

Наибольшее по модулю собственное число, полученное методом скалярных произведений: -1.407780
Соответствующий ему собственный вектор: DenseVector 3-Double
 0,696591
 0,121497
-0,707106

Фактическая погрешнсть: 0.000000


index,AYminusLambdaY,AbsFactError,K,Lambda,PosteriorEstimation
0,1.083790880705197,0.587836786281753,1,-0.8199428283921549,0.7069198381591515
1,0.6779524153390699,0.17575068288362306,2,-1.2320289317902848,0.3253078391090966
2,0.4361676138360931,0.043618718868072115,3,-1.3641608958058358,0.1499026282533117
3,0.2921397140951521,0.010328220823764056,4,-1.3974513938501438,0.07145740410978563
4,0.1983090543737116,0.002423291875353728,5,-1.4053563227985542,0.03447139823375866
5,0.13508543591219951,0.0005680058784465203,6,-1.4072116087954614,0.01668151879310448
6,0.09210093686355766,0.0001331882479431723,7,-1.4076464264259647,0.008079175709926346
7,0.0628098623702309,3.1243699033511874E-05,8,-1.4077483709748744,0.003913800306886272
8,0.042837708915344434,7.331291391077244E-06,9,-1.4077722833825168,0.0018961075516774506
9,0.029217144056296773,1.7205593854097145E-06,10,-1.4077778941145225,0.0009186285930617699
